In [ ]:
# This script will look for file_fastq items and check if they have quality metric, but lack the related workflow_run
# It will delete the quality_metric object and also clear the quality_metric field in the file-fastq

from core.utils import Tibanna
from core import ff_utils
from datetime import datetime

# set enviroment and key/connection
env = 'fourfront-webdev'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

"elasticbeanstalk-fourfront-webdev-files"

def get_wfr_report(wfr):
    """For a given workflow_run_sbg item, grabs details, uuid, run_status, wfr name, date, and run time"""
    wfr_data= ff_utils.get_metadata(wfr , connection=ff)
    wfr_uuid = wfr_data['uuid']
    wfr_status = wfr_data['run_status']
    wfr_name = wfr_data['title'].split(' run ')[0]
    wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
    run_hours = (datetime.now()-wfr_time).total_seconds()/3600
    wfr_rep = {'wfr_uuid': wfr_data['uuid'],
               'wfr_status': wfr_data['run_status'],
               'wfr_name': wfr_data['title'].split(' run ')[0],
               'wfr_date': wfr_time,
               'run_time': run_hours}
    return wfr_rep

fastqc_status =[]
# get all files for fastqc
files = ff_utils.get_metadata('files-fastq' , connection=ff)['@graph']
found_any = False
for raw_file in files:
    if raw_file.get('quality_metric'):
        wfr_report = []
        wfrs = raw_file.get('workflow_run_inputs')
        # get a report on all workflow_runs
        if wfrs:
            for wfr in wfrs:
                wfr_report.append(get_wfr_report(wfr))  
        # sort the report by date and name
        wfr_report = sorted(wfr_report, key=lambda k: (k['wfr_date'], k['wfr_name']))
        sub_list = [i for i in wfr_report if i['wfr_name'] == 'fastqc-0-11-4-1/1']
        if not sub_list:
            if not found_any:
                print "The following files have a quality metric, but no fastqc, the quality metric is deleted"
                found_any = True
            print raw_file['uuid']
            # change quality_metric status to deleted
            qc_data = ff_utils.get_metadata(raw_file['quality_metric'], connection=ff)
            # some old qc items have a string value in int form, correct during patch
            if isinstance(qc_data.get('Sequence length'), int):
                patch_data = {'status': 'deleted', 'Sequence length': str(qc_data.get('Sequence length'))}
            else:    
                patch_data = {'status': 'deleted'}
            ff_utils.patch_metadata(patch_data, obj_id=raw_file['quality_metric'] ,connection=ff)
            ff_utils.delete_field(raw_file, 'quality_metric', connection=ff)

if not found_any:
    print "no file_fastq with quality_metric missing wf_run found"

print 'Done'
         